# RNN과 어텐션을 사용한 자연어 처리
* 자연어에 많이 사용되는건 순환 신경망이다.
* 어텐션 메커니즘과 트랜스포머에 알아본다.
---
## char-rnn을 사용해 셰익스피어 문체 생성하기
* 2015년 RNN을 훈련하여 문장에서 다음 글자를 예측하는 방법을 소개했다.
* 모델이 다음 글자를 예측하도록훈련한 것 만으로 단어,문법,적절한 구두점 등을 학습했다는 점!
#### 훈련 데이터셋 만들기

In [ ]:
import tensorflow as tf
from tensorflow import keras
import sklearn
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)# url 받아오기
with open(filepath) as f:
    shakespeare_text = f.read()

In [ ]:
print(shakespeare_text[:148]) # 148번까지 출력

* 모든 글자를 정수로 인코딩한다.
* 텍스트에 사용되는 모든 글자를 다른 글자 ID에 매핑한다.
* char_level=True 로 설정해서 글자 수준 인코딩을 만든다.
* 기본적으로 텍스트를 소문자로 바꾼다. lower=False로 조정 가능


In [ ]:
tokenizer=keras.preprocessing.text.Tokenizer(char_level=True) 
tokenizer.fit_on_texts(shakespeare_text)

* 문장을 글자ID로 인코딩 하거나 디코딩 할 수 있다.(글자 ID는 1부터 시작한다)
* 이를 통해 텍스트에 있는 고유 글자 개수와 전체 글자 개수를 알 수 있다.

In [ ]:
print(tokenizer.texts_to_sequences(["First"]))
print(tokenizer.sequences_to_texts([[20,6,9,8,3]]))
max_id=len(tokenizer.word_index)# 고유 글자 개수
dataset_size=tokenizer.document_count# 전체 글자 개수

* 각 글자를 ID로 나타내어 본다(1빼야됨)

In [ ]:
[encoded]=np.array(tokenizer.texts_to_sequences([shakespeare_text]))-1

### 순차 데이터셋을 나누는 방법
* 훈련,검증,테스트 중복 안되는게 중요하다.
* 시계열은 보통 시간에 따라 나눈다.
* RNN은 과거가 미래에서 잘 바뀌지 않는다 가정한다. 이는 많은 경우 타당하다
* 그렇지않은 금융시장 등이 있다.
* 시계열이 안정적인지 확인하려면 검증 세트에 대한 모델의 오차를 그려보면 된다.
* 검증세트 마지막보다 첫 부분에서 성능이 더 좋다면 안정되지 않을 수 있다.

In [ ]:
train_size=dataset_size*90//100 #90% 트레인셋
dataset=tf.data.Dataset.from_tensor_slices(encoded[:train_size]) # 이 세트에서 한 번에 한 글자씩 반환하는 객체 생성

### 순차 데이터셋을 윈도 여러개로 자르기
* 훈련세트는 백만 개 이상의 글자로 이루어진 시퀀스이다.
* 이걸 직접 신경망을 훈련시킬 수는 없다.
  * 이 RNN은 백만 개의 층이 있는 심층 신경망과 비슷하고 샘플 하나로 훈련하는 셈이 된다.
* 데이터셋의 window() 메서드를 통해 이 긴 시퀀스를 작은 많은 텍스트 윈도로 변환한다.
* 각 샘플은 전체 텍스트에서 매우 짧은 부분 문자열이다.
* RNN은 이 부분 문자열 길이만큼만 역전파를 위해 펼쳐진다. 이를 TBPTT라고 부른다.

In [ ]:
n_steps=100
window_length=n_steps+1
dataset=dataset.window(window_length,shift=1,drop_remainder=True) # 기본적으로 window는 중복하지 않는다
# shift=1로 지정하면 가장 큰 훈련세트 만들 수 있음 stride=1처럼

* 윈도는 중첩 데이터셋이다 리스트와 비슷
* 이를 플렛 데이터셋을 ㅗ변경해야한다.


In [ ]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

* 경사하강법은 훈련 세트 샘플이 동일 독립분포 일 때 가장 잘 작동하기에 윈도를 섞는다.


In [ ]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [ ]:
dataset = dataset.map( 
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)) # 원핫 벡터나 임베딩으로 인코딩 되어야한다.

In [ ]:
dataset=dataset.prefetch(1)

### Char-RNN 모델 만들고 훈련하기